In [0]:
%matplotlib inline
import torch as t
from torch import nn
from torch.autograd import Variable
from torch.optim import Adam
from torchvision import transforms
from torchvision.utils import make_grid
from torchvision.datasets import CIFAR10
# from pylab import plt
import matplotlib.pyplot as plt
import h5py
import numpy as np

OneHot

In [0]:
# 输入为一维行向量
def one_hot(x,mu=256): # 1*250
  hot = np.zeros((mu,x.shape[0]))
  for i in np.arange(x.shape[0]):
      hot[x[i],i]=1
  return hot     

Encode & Decode

In [0]:
def encode_mu_law(x, mu=256):
    mu = mu-1
    fx = np.sign(x)*np.log(1+mu*np.abs(x))/np.log(1+mu)
    return np.floor((fx+1)/2*mu+0.5).astype(np.long)

def decode_mu_law(y, mu=256):  
    mu = mu-1
    fx = (y-0.5)/mu*2-1
    x = np.sign(fx)/mu*((1+mu)**np.abs(fx)-1)
    return x

Data preprocess

In [0]:
def load_data():
  f = h5py.File("drive/DCGAN-ECG/ecg_data.h5",'r')
  ecg_train = f['ecg_train']
  ecg_test = f['ecg_test']
  return ecg_train,ecg_test

In [11]:
mu=64
# 读取数据
ecg_train,ecg_test = load_data()
# print(ecg_train[:10,:10])

# 数据编码
y = encode_mu_law(ecg_train[:,:64],mu=mu)    # 每个样本只取前64个样本点
plt.plot(y[0][:])
# print(y.shape)
# print(y[:10,:10])

# onehot编码
hot_y = np.zeros((y.shape[0],mu,y.shape[1]))  # 实例个数*mu*采样点个数
for i in np.arange(hot_y.shape[0]):
  hot_y[i] = one_hot(y[i][:],mu = mu)

# print(hot_y.shape)
# print(hot_y[:5,:10,:10])


OSError: ignored

In [12]:
# 授权验证
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!wget https://launchpad.net/~alessandro-strada/+archive/ubuntu/google-drive-ocamlfuse-beta/+build/15331130/+files/google-drive-ocamlfuse_0.7.0-0ubuntu1_amd64.deb
!dpkg -i google-drive-ocamlfuse_0.7.0-0ubuntu1_amd64.deb
!apt-get install -f
!apt-get -y install -qq fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}


E: Package 'python-software-properties' has no installation candidate
--2018-12-12 00:34:28--  https://launchpad.net/~alessandro-strada/+archive/ubuntu/google-drive-ocamlfuse-beta/+build/15331130/+files/google-drive-ocamlfuse_0.7.0-0ubuntu1_amd64.deb
Resolving launchpad.net (launchpad.net)... 91.189.89.222, 91.189.89.223
Connecting to launchpad.net (launchpad.net)|91.189.89.222|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2018-12-12 00:34:30 ERROR 404: Not Found.

dpkg: error: cannot access archive 'google-drive-ocamlfuse_0.7.0-0ubuntu1_amd64.deb': No such file or directory
Reading package lists... Done
Building dependency tree       
Reading state information... Done
0 upgraded, 0 newly installed, 0 to remove and 8 not upgraded.
··········
/bin/bash: google-drive-ocamlfuse: command not found


In [0]:
# 指定Google Drive云端硬盘的根目录，名为drive
!mkdir -p drive
!google-drive-ocamlfuse drive

DCGAN config

In [0]:
class Config:
    lr = 0.0002 #learning rate
    nz = 100 # noise dimension
    image_size = 64
    image_size2 = 64 
    nc = 1 # chanel of img 
    ngf = 64 # generate channel
    ndf = 64 # discriminative channel
    beta1 = 0.5
    batch_size = 64  
#     max_epoch = 10 # =1 when debug
    max_epoch = 100 # =1 when debug
    workers = 2  #加载数据的子进程数，0时只有主进程
    gpu = True  # use gpu or not
    
opt=Config()

make_tensor

In [0]:
def make_tensor():
  y = hot_y.reshape(hot_y.shape[0],-1,hot_y.shape[1],hot_y.shape[2])
  # z=np.concatenate((y,y,y),axis=1)              # 单通道到三通道
  data = t.from_numpy(y/1.)

  
  torch_data = t.utils.data.TensorDataset(data,data)  #target不重要，没有用
  dataloader = t.utils.data.DataLoader(torch_data,opt.batch_size,shuffle=True,num_workers=opt.workers)  
  return dataloader

# make_tensor()

demake_tensor

In [0]:
# torch.Size([32, 3, 64, 64])
def demake_tensor(x):
#   y = x.numpy()
  y = x.detach().cpu().numpy()
  N = y.shape[0]
  mu = y.shape[2]
  num_nodes = y.shape[3]
#   z = np.sum(y,axis=1) 
  z = y.reshape(N,mu,-1)
  assert z.shape==(N,mu,num_nodes),"dimension is wrong."
#   return z/3.
  return z

find_maximal

In [0]:
# (num ,mu , num_nodes) --> (num , num_nodes)
def find_maximal(x):
  N = x.shape[0]
  num_nodes = x.shape[2]
  prob = np.zeros((N,1,num_nodes))
  prob = np.max(x,axis=1)
  return prob

Generator

In [0]:
netg = nn.Sequential(
    nn.ConvTranspose2d(opt.nz,opt.ngf*8,4,1,0,bias=False),  
    nn.BatchNorm2d(opt.ngf*8),
    nn.ReLU(True),

    nn.ConvTranspose2d(opt.ngf*8,opt.ngf*4,4,2,1,bias=False),
    nn.BatchNorm2d(opt.ngf*4),
    nn.ReLU(True),

    nn.ConvTranspose2d(opt.ngf*4,opt.ngf*2,4,2,1,bias=False),
    nn.BatchNorm2d(opt.ngf*2),
    nn.ReLU(True),

    nn.ConvTranspose2d(opt.ngf*2,opt.ngf,4,2,1,bias=False),
    nn.BatchNorm2d(opt.ngf),
    nn.ReLU(True),

    nn.ConvTranspose2d(opt.ngf,opt.nc,4,2,1,bias=False),
    nn.Tanh()
)

Discriminator


In [0]:
netd = nn.Sequential(
    nn.Conv2d(opt.nc,opt.ndf,4,2,1,bias=False),
    nn.LeakyReLU(0.2,inplace=True),

    nn.Conv2d(opt.ndf,opt.ndf*2,4,2,1,bias=False),
    nn.BatchNorm2d(opt.ndf*2),
    nn.LeakyReLU(0.2,inplace=True),

    nn.Conv2d(opt.ndf*2,opt.ndf*4,4,2,1,bias=False),
    nn.BatchNorm2d(opt.ndf*4),
    nn.LeakyReLU(0.2,inplace=True),

    nn.Conv2d(opt.ndf*4,opt.ndf*8,4,2,1,bias=False),
    nn.BatchNorm2d(opt.ndf*8),
    nn.LeakyReLU(0.2,inplace=True),

    nn.Conv2d(opt.ndf*8,1,4,1,0,bias=False),
    nn.Sigmoid()
)

Optimizer

In [0]:
# optimizerD = Adam(netd.parameters(),lr=opt.lr,betas=(opt.beta1,0.999))
# optimizerG = Adam(netg.parameters(),lr=opt.lr,betas=(opt.beta1,0.999))

optimizerD = Adam(netd.parameters(),lr=opt.lr,betas=(opt.beta1,0.5))   # 论文建议0.5
optimizerG = Adam(netg.parameters(),lr=opt.lr,betas=(opt.beta1,0.5))

# criterion
criterion = nn.BCELoss()    # compute loss

fix_noise = Variable(t.FloatTensor(opt.batch_size,opt.nz,1,1).normal_(0,1))
if opt.gpu:
    fix_noise = fix_noise.cuda()
    netd.cuda()
    netg.cuda()
    criterion.cuda() # it's a good habit ，使用cuda计算损失

Training

In [0]:
# begin training
dataloader = make_tensor()

print('Start training......')

loss_D = np.zeros(opt.max_epoch)
loss_G = np.zeros(opt.max_epoch)

for epoch in range(opt.max_epoch):
    for ii,data in enumerate(dataloader,0):  # ii is step
#         print("Tringing.....")
        real,_ = data
        input = Variable(real)  # batch_size,channels,width,height
        label = Variable(t.ones(input.size(0))) # 1 for real
        noise = t.randn(input.size(0),opt.nz,1,1)  # opt.nz=100
        noise = Variable(noise)
        
        if opt.gpu:
            noise = noise.cuda()
            input = input.cuda()
            label = label.cuda()
        
        # ----- train netd -----
        netd.zero_grad()
        ## train netd with real img    
        ## Input type (torch.cuda.DoubleTensor) and weight type (torch.cuda.FloatTensor) should be the same
        output=netd(input.float())  
        error_real=criterion(output.squeeze(),label)    
        error_real.backward()
        D_x=output.data.mean()
        ## train netd with fake img
        fake_pic=netg(noise).detach()
        output2=netd(fake_pic)
        label.data.fill_(0) # 0 for fake
        error_fake=criterion(output2.squeeze(),label)
        error_fake.backward()
        D_x2=output2.data.mean()
        error_D=error_real+error_fake
        optimizerD.step()
        
        # ------ train netg -------
        netg.zero_grad()
        label.data.fill_(1)
        noise.data.normal_(0,1)
        fake_pic=netg(noise)
        output=netd(fake_pic)
        error_G=criterion(output.squeeze(),label)
        error_G.backward()
        optimizerG.step()
        D_G_z2=output.data.mean()
        
        loss_D[epoch]+=error_D.data[0]
        loss_G[epoch]+=error_G.data[0] 
        
    if epoch % 20 ==0:
        print("epoch=",epoch)
#         print('{epoch}  lossD:{loss_D},lossG:{loss_G}'.format(
#                epoch=epoch,loss_D=loss_D,loss_G=loss_G))
        fake_u=netg(fix_noise)
        img1 = demake_tensor(fake_u)
        img2 = find_maximal(img1)
        ecg = decode_mu_law(img2,64)
        print(ecg[0,:5])
        plt.plot(ecg[0,:])        
        plt.show()
        
plt.plot(loss_D,'r--',label="Discriminator")
plt.plot(loss_D,'b--',label="Generator")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("How Loss Changed")
plt.lengend(loc="upper left")
plt.show